In [6]:
import numpy as np
from moabb.datasets import BNCI2014002
import pandas as pd
import mne
from moabb.paradigms import  MotorImagery

%matplotlib inline

In [7]:
dataset = BNCI2014002()
paradigm = MotorImagery(n_classes=2)

def get_raw(dataset, paradigm, n_channels, sfreq):
    '''
    Takes a dataset from moabb datasets and creates 3 dictionaries with subject number as keys and X matrix, y labels and metadata
    as dictionary values respectively.
    X: Original 3D data matrix (num_cases, n_channels, lectures) is reshaped to 2D matrix ((num_cases*lectures),n_channels)
    y: This array corresponds with the class for each case. It has num_cases length.
    metadata: Describes each subject
    '''
    
    subjects_all = dataset.subject_list
    channels = [str(a+1) for a in range(n_channels)]
    

    X_all_subjects = {}
    y_all_subjects = {}
    metadata_all_subjects = {}
    
    for subject in subjects_all:
        
        X, y, metadata = paradigm.get_data(dataset=dataset, subjects=[subject])
        #X_2d = X.reshape((X.shape[0]*X.shape[2]),n_channels)
        #data_mne = X_2d.transpose()
        data_mne = X / 1000000
        
                        
        #info_mne = mne.create_info(
            #ch_names=channels,
            #ch_types='eeg',
            #sfreq=sfreq)
        
        #raw = mne.io.RawArray(data_mne, info_mne)
        #ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
        #raw_1020 = raw.copy().set_montage(ten_twenty_montage)

        X_all_subjects.update({str(subject):data_mne})
        y_all_subjects.update({str(subject):y})
        metadata_all_subjects.update({str(subject):metadata})
        
    return X_all_subjects, y_all_subjects, metadata_all_subjects
        
        
X_all_subjects, y_all_subjects, metadata_all_subjects = get_raw(dataset, paradigm, 15, 512)

print(X_all_subjects)



Choosing from all possible events


{'1': array([[[-3.69933537e-06, -2.85635093e-06, -1.96606254e-06, ...,
          2.38058237e-06,  1.52865361e-06,  6.22539578e-07],
        [-3.63668614e-06, -2.82721729e-06, -1.98717082e-06, ...,
          6.44032852e-07, -4.98763642e-07, -1.66368852e-06],
        [-4.12123709e-06, -3.20450810e-06, -2.19647161e-06, ...,
          7.10352006e-07, -1.28114374e-07, -9.40015688e-07],
        ...,
        [-4.43718715e-06, -2.87902534e-06, -1.35013671e-06, ...,
          6.57667597e-07,  7.89893999e-08, -4.12774599e-07],
        [-6.08738801e-06, -4.67519376e-06, -3.21054857e-06, ...,
          1.25227674e-06,  2.80881222e-07, -6.70324682e-07],
        [-5.41832853e-06, -4.20571010e-06, -2.89895877e-06, ...,
          7.11359224e-08, -6.20440040e-07, -1.25280068e-06]],

       [[-3.19029353e-06, -2.99171532e-06, -2.61733818e-06, ...,
         -9.41137533e-07, -1.23279279e-06, -1.48193362e-06],
        [-5.33046588e-06, -4.41605751e-06, -3.27766388e-06, ...,
         -5.12343373e-07, -1.472

In [28]:
data_epochs_1 = X_all_subjects['1'][:,:,:2560]
data_epochs_1 = data_epochs_1.reshape(2560,15,160)
print(data_epochs_1.shape)

(2560, 15, 160)


In [31]:
X = get_features(data_epochs_1, 512)
Y = y_all_subjects['1']
X.shape

ValueError: too many values to unpack (expected 2)

In [26]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import pandas as pd


def epoch_data(data, window_length, overlap):
    arr = []
    i = 0
    while i + window_length < len(data):
        arr.append(data[i:i+window_length])
        i += overlap
    return np.array(arr)
def get_features(data_epochs, sampling_freq):
    num_epochs, win_length = data_epochs.shape
    Y = fft.fft(data_epochs)/win_length
    PSD = 2*np.abs(Y[0:int(win_length/2), :])
    f = sampling_freq/2*np.linspace(0, 1, int(win_length/2))
    # SPECTRAL FEATURES
    # Average of band powers
    # Delta <4
    ind_delta, = np.where(f < 4)
    meanDelta = np.mean(PSD[:, ind_delta], axis = -1)
    # Theta 4-8
    ind_theta, = np.where((f >= 4) & (f <= 8))
    meanTheta = np.mean(PSD[:, ind_theta], axis = -1)
    # Alpha 8-12
    ind_alpha, = np.where((f >= 8) & (f <= 12))
    meanAlpha = np.mean(PSD[:, ind_alpha], axis = -1)
    # Beta 12-30
    ind_beta, = np.where((f >= 12) & (f < 30))
    meanBeta = np.mean(PSD[:, ind_beta], axis=-1)
    feature_vector = np.array([meanDelta, meanTheta, meanAlpha,
                                     meanBeta]).T
    feature_vector = np.log10(feature_vector)
    feature_vector_shape = feature_vector.shape
    return feature_vector

def draw_specgram(ch, fs_Hz, fig, i):
    NFFT = fs_Hz*2
    overlap = NFFT - int(0.25 * fs_Hz)
    spec_PSDperHz, spec_freqs, spec_t = mlab.specgram(np.squeeze(ch),
                                   NFFT=NFFT,
                                   window=mlab.window_hanning,
                                   Fs=fs_Hz,
                                   noverlap=overlap
                                   ) # returns PSD power per Hz
    # convert the units of the spectral data
    
    spec_PSDperBin = spec_PSDperHz * fs_Hz / float(NFFT)
    f_lim_Hz = [0, 60]   # frequency limits for plotting
    #plt.figure(figsize=(10,5))
    ax = fig.add_subplot(2,1,i)
    plt.pcolor(spec_t, spec_freqs, 10*np.log10(spec_PSDperBin))  # dB re: 1 uV
    plt.clim([-25,26])
    plt.xlim(spec_t[0], spec_t[-1]+1)
    plt.ylim(f_lim_Hz)
    plt.xlabel('Time (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.show()
    
def compute_feature_vector(eegdata, fs):
    """Extract the features from the EEG.
    Args:
        eegdata (numpy.ndarray): array of dimension [number of samples,
                number of channels]
        fs (float): sampling frequency of eegdata
    Returns:
        (numpy.ndarray): feature matrix of shape [number of feature points,
            number of different features]
    """
    # 1. Compute the PSD
    winSampleLength, nbCh = eegdata.shape

    # Apply Hamming window
    w = np.hamming(winSampleLength)
    dataWinCentered = eegdata - np.mean(eegdata, axis=0)  # Remove offset
    dataWinCenteredHam = (dataWinCentered.T*w).T

    NFFT = nextpow2(winSampleLength)
    Y = np.fft.fft(dataWinCenteredHam, n=NFFT, axis=0)/winSampleLength
    PSD = 2*np.abs(Y[0:int(NFFT/2), :])
    f = fs/2*np.linspace(0, 1, int(NFFT/2))

    # SPECTRAL FEATURES
    # Average of band powers
    # Delta <4
    ind_delta, = np.where(f < 4)
    meanDelta = np.mean(PSD[ind_delta, :], axis=0)
    # Theta 4-8
    ind_theta, = np.where((f >= 4) & (f <= 8))
    meanTheta = np.mean(PSD[ind_theta, :], axis=0)
    # Alpha 8-12
    ind_alpha, = np.where((f >= 8) & (f <= 12))
    meanAlpha = np.mean(PSD[ind_alpha, :], axis=0)
    # Beta 12-30
    ind_beta, = np.where((f >= 12) & (f < 30))
    meanBeta = np.mean(PSD[ind_beta, :], axis=0)

    feature_vector = np.concatenate((meanDelta, meanTheta, meanAlpha,
                                     meanBeta), axis=0)

    feature_vector = np.log10(feature_vector)

    return feature_vector


def nextpow2(i):
    """
    Find the next power of 2 for number i
    """
    n = 1
    while n < i:
        n *= 2
    return n


def compute_feature_matrix(epochs, fs):
    """
    Call compute_feature_vector for each EEG epoch
    """
    n_epochs = epochs.shape[2]

    for i_epoch in range(n_epochs):
        if i_epoch == 0:
            feat = compute_feature_vector(epochs[:, :, i_epoch], fs).T
            feature_matrix = np.zeros((n_epochs, feat.shape[0])) # Initialize feature_matrix

        feature_matrix[i_epoch, :] = compute_feature_vector(
                epochs[:, :, i_epoch], fs).T

    return feature_matrix


In [27]:

fname_ec = '../data/EyesClosedNTXDemo.txt' 
fname_eo = '../data/EyesOpenedNTXDemo.txt' 
data_ec = np.loadtxt(fname_ec,
                  delimiter=',',
                  skiprows=7,
                  usecols=(1))
data_ec_shape = data_ec.shape
data_eo = np.loadtxt(fname_eo,
                  delimiter=',',
                  skiprows=7,
                  usecols=(1))
sampling_freq = 250
window_size = int(3 * sampling_freq)
window_overlap = int(0.5 * sampling_freq)
fig = plt.figure()
draw_specgram(data_eo, sampling_freq, fig, 1)
draw_specgram(data_ec, sampling_freq, fig, 2)

data_epochs_ec = epoch_data(data_ec, window_size, window_overlap)
data_epochs_ec_shape = data_epochs_ec.shape
data_epochs_eo = epoch_data(data_eo, window_size, window_overlap)

ec_features = get_features(data_epochs_ec, sampling_freq)
eo_features = get_features(data_epochs_eo, sampling_freq)

ec_features = np.hstack((ec_features, np.ones([ec_features.shape[0],1])))
eo_features = np.hstack((eo_features, np.zeros([eo_features.shape[0],1])))

X = np.vstack((ec_features, eo_features))[:,:-1]
Y = np.vstack((ec_features, eo_features))[:,-1]
validation_size = 0.2
seed = 7

X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB(var_smoothing=0.001)))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

    
''''
fname_eo_test = '../data/OpenBCI-RAW-2018-04-17_17-21-25.txt'
fname_ec_test = '../data/Ganglion1minEyesOpen-1minEyesClosed.txt'

eo_test = np.loadtxt(fname_eo_test,
                  delimiter=',',
                  skiprows=7,
                  usecols=(1))
ec_test = np.loadtxt(fname_ec_test,
                  delimiter=',',
                  skiprows=sampling_freq*10,    # skip first 10 seconds (noisy)
                  usecols=(3))                  # channel 3 is at O1

ec_test = ec_test[70*sampling_freq : 100*sampling_freq]             # grab seconds 70 to 100 

fig = plt.figure()
draw_specgram(eo_test, sampling_freq, fig, 1)
draw_specgram(ec_test, sampling_freq, fig, 2)

data_epochs_ec_test = epoch_data(ec_test, window_size, window_overlap)
data_epochs_eo_test = epoch_data(eo_test, window_size, window_overlap)

ec_test_features = get_features(data_epochs_ec_test, sampling_freq)
eo_test_features = get_features(data_epochs_eo_test, sampling_freq)

Y_eo_test = [0]*len(eo_test_features)
Y_ec_test = [1]*len(ec_test_features)

for name, model in models:
    model.fit(X_train, Y_train)
    ec_test_score = model.score(ec_test_features, Y_ec_test)
    eo_test_score = model.score(eo_test_features, Y_eo_test)
    validation_score = model.score(X_validation, Y_validation)
    msg = "%s: validation %f, test (closed) %f, test (open) %f" % (name, validation_score, ec_test_score, eo_test_score)
    print(msg)

'''

ValueError: Found input variables with inconsistent numbers of samples: [2560, 160]